# Homework and bake-off: pragmatic color descriptions

In [ ]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Summer 2021"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [All two-word examples as a dev corpus](#All-two-word-examples-as-a-dev-corpus)
1. [Dev dataset](#Dev-dataset)
1. [Random train–test split for development](#Random-train–test-split-for-development)
1. [Question 1: Improve the tokenizer [1 point]](#Question-1:-Improve-the-tokenizer-[1-point])
1. [Use the tokenizer](#Use-the-tokenizer)
1. [Question 2: Improve the color representations [1 point]](#Question-2:-Improve-the-color-representations-[1-point])
1. [Use the color representer](#Use-the-color-representer)
1. [Initial model](#Initial-model)
1. [Question 3: GloVe embeddings [1 point]](#Question-3:-GloVe-embeddings-[1-point])
1. [Try the GloVe representations](#Try-the-GloVe-representations)
1. [Question 4: Color context [3 points]](#Question-4:-Color-context-[3-points])
1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])
1. [Submission Instruction](#Submission-Instruction)

## Overview

This homework and associated bake-off are oriented toward building an effective system for generating color descriptions that are pragmatic in the sense that they would help a reader/listener figure out which color was being referred to in a shared context consisting of a target color (whose identity is known only to the describer/speaker) and a set of distractors.

The notebook [colors_overview.ipynb](colors_overview.ipynb) should be studied before work on this homework begins. That notebook provides backgroud on the task, the dataset, and the modeling code that you will be using and adapting.

The homework questions are more open-ended than previous ones have been. Rather than asking you to implement pre-defined functionality, they ask you to try to improve baseline components of the full system in ways that you find to be effective. As usual, this culminates in a prompt asking you to develop a novel system for entry into the bake-off. In this case, though, the work you do for the homework will likely be directly incorporated into that system (not required, but an efficient way to work at the very least).

## Set-up

See [colors_overview.ipynb](colors_overview.ipynb) for set-up in instructions and other background details.

In [1]:
from colors import ColorsCorpusReader
from nltk.translate.bleu_score import corpus_bleu
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torch_color_describer import ContextualColorDescriber
from torch_color_describer import create_example_dataset

import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL

In [2]:
utils.fix_random_seeds()

In [3]:
COLORS_SRC_FILENAME = os.path.join(
    "data", "colors", "filteredCorpus.csv")

## All two-word examples as a dev corpus

So that you don't have to sit through excessively long training runs during development, I suggest working with the two-word-only subset of the corpus until you enter into the late stages of system testing.

In [4]:
dev_corpus = ColorsCorpusReader(
    COLORS_SRC_FILENAME,
    word_count=2,
    normalize_colors=True)

In [5]:
dev_examples = list(dev_corpus.read())

This subset has about one-third the examples of the full corpus:

In [6]:
len(dev_examples)

13890

We __should__ worry that it's not a fully representative sample. Most of the descriptions in the full corpus are shorter, and a large proportion are longer. So this dataset is mainly for debugging, development, and general hill-climbing. All findings should be validated on the full dataset at some point.

## Dev dataset

The first step is to extract the raw color and raw texts from the corpus:

In [6]:
dev_rawcols, dev_texts = zip(*[[ex.colors, ex.contents] for ex in dev_examples])

The raw color representations are suitable inputs to a model, but the texts are just strings, so they can't really be processed as-is. Question 1 asks you to do some tokenizing!

## Random train–test split for development

For the sake of development runs, we create a random train–test split:

In [7]:
dev_rawcols_train, dev_rawcols_test, dev_texts_train, dev_texts_test = \
    train_test_split(dev_rawcols, dev_texts)

## Question 1: Improve the tokenizer [1 point]

This is the first required question – the first required modification to the default pipeline.

The function `tokenize_example` simply splits its string on whitespace and adds the required start and end symbols:

In [190]:
"""
Tokenizer approach overview:
DEL: ADD MORE
- Handle casing
- Remove word endings (-er, -est, -ish) as described in Monroe 2017.
- To better align tokens and word representations, we want tokens
  that are similar to GloVe's. 
- More complex tokenization schemes are likely not needed, since
  it appears that comparatively simple words are used.
"""
!pip install -U nltk

from nltk.parse import CoreNLPParser
from transformers import BertModel, BertTokenizer

# DEL: Original
def tokenize_example_orig(s):
    return [START_SYMBOL] + s.split() + [END_SYMBOL]

bert_tok = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_example(s):
    #s = s.lower()
    return [START_SYMBOL] + bert_tok.tokenize(s) + [END_SYMBOL]
    """
    # Stanford's tokenizer, which is also used for GloVe.
    # Assumes the server is set-up and running.
    parser = CoreNLPParser(url='http://localhost:9000')
    words = list(bert_tok.tokenize(s))
    #words = list(parser.tokenize(s))
    segments = []
    i = 0
    for word in words:
        ending_len = 0
        if word.endswith('er'):
            ending_len = -2
        elif word.endswith('est') or word.endswith('ish'):
            ending_len = -3
        segments.extend([word] if ending_len == 0 else [word[:ending_len], word[ending_len:]])
    
    return [START_SYMBOL] + [s for s in segments] + [END_SYMBOL]
    """ 
        

In [98]:
# DEL:
print(tokenize_example_orig(dev_texts_train[376]))
print(tokenize_example("Foo ### bar ASDF!?"))
print(tokenize_example("Greenest of the greenish green"))
# NEW:
print(tokenize_example(dev_texts_train[376]))

['<s>', 'aqua,', 'teal', '</s>']
['<s>', 'foo', '#', '#', '#', 'bar', 'as', '##df', '!', '?', '</s>']
['<s>', 'greene', '##st', 'of', 'the', 'greenish', 'green', '</s>']
['<s>', 'aqua', ',', 'tea', '##l', '</s>']


__Your task__: Modify `tokenize_example` so that it does something more sophisticated with the input text. 

__Notes__:

* There are useful ideas for this in [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142)
* There is no requirement that you do word-level tokenization. Sub-word and multi-word are options.
* This question can interact with the size of your vocabulary (see just below), and in turn with decisions about how to use `UNK_SYMBOL`.

__Important__: don't forget to add the start and end symbols, else the resulting models will definitely be terrible! The following test will check that your tokenizer has this property:

In [12]:
def test_tokenize_example(func):
    s = "A test string"
    result = func(s)
    assert all(isinstance(tok, str) for tok in result), \
        "The tokenizer must return a list of strings."
    assert result[0] == START_SYMBOL, \
        "The tokenizer must add START_SYMBOL as the first token."
    assert result[-1] == END_SYMBOL, \
        "The tokenizer must add END_SYMBOL as the final token."

In [32]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_tokenize_example(tokenize_example)

## Use the tokenizer

Once the tokenizer is working, run the following cell to tokenize your inputs:

In [191]:

dev_seqs_train = [tokenize_example(s) for s in dev_texts_train]

dev_seqs_test = [tokenize_example(s) for s in dev_texts_test]
"""

# DEL:
print(len(dev_texts_train))
i = 0
dev_seqs_train = []
for s in dev_texts_train:
    if i % 10 == 0:
        print("{0}/{1}: {2}".format(i, len(dev_texts_train), s))
    dev_seqs_train.append(tokenize_example(s)) 
    i += 1
   
i = 0
dev_seqs_test = []
for s in dev_texts_test:
    if i % 10 == 0:
        print("{0}/{1}: {2}".format(i, len(dev_texts_test), s))
    i += 1
    dev_seqs_test.append(tokenize_example(s)) 
i = 0
"""

'\n\n# DEL:\nprint(len(dev_texts_train))\ni = 0\ndev_seqs_train = []\nfor s in dev_texts_train:\n    if i % 10 == 0:\n        print("{0}/{1}: {2}".format(i, len(dev_texts_train), s))\n    dev_seqs_train.append(tokenize_example(s)) \n    i += 1\n   \ni = 0\ndev_seqs_test = []\nfor s in dev_texts_test:\n    if i % 10 == 0:\n        print("{0}/{1}: {2}".format(i, len(dev_texts_test), s))\n    i += 1\n    dev_seqs_test.append(tokenize_example(s)) \ni = 0\n'

We use only the train set to derive a vocabulary for the model:

In [193]:
dev_vocab = sorted({w for toks in dev_seqs_train for w in toks})

dev_vocab += [UNK_SYMBOL]

It's important that the `UNK_SYMBOL` is included somewhere in this list. In test examples, words not seen in training will be mapped to `UNK_SYMBOL`. 

Conceptual note: If you model's vocab is the same as your train vocab, then `UNK_SYMBOL` will never be encountered during training, so it will be a random vector at test time.

In [31]:
len(dev_vocab)

924

## Question 2: Improve the color representations [1 point]

This is the second required pipeline improvement for the assignment. 

The following functions do nothing at all to the raw input colors we get from the corpus. 

In [194]:
def represent_color_context(colors):
    # Improve me!
    # DEL: TODO
    colors = np.array(colors)
    target = colors[-1]
    colors = [target - colors[0], target - colors[1], target]
    return [represent_color(color) for color in colors]


def represent_color(color):
    # HLS to HSV - 0-indexed from L to R.
    h = color[0]  # Stays the same.
    sat_hls = color[1]
    light = color[2]
    
    sat_tmp = sat_hls * (light if light < 0.5 else 1.0 - light)
    v = light + sat_tmp
    s = 2.0 * sat_tmp / v if v > 0 else 0
    
    # Fourier transform, as per Monroe et al. in 
    # "Learning to Generate Compositional Color Descriptions"
    # HSV are already normalized to 1.0.
    f_hat_re = []
    f_hat_im = []
    for j in range(3):
        for k in range(3):
            for l in range(3):
                f_jkl = np.exp(-2j * np.pi * (j * h + k * s + l * v))
                f_hat_re.append(f_jkl.real)
                f_hat_im.append(f_jkl.imag)
    return f_hat_re + f_hat_im

In [36]:
represent_color_context(dev_rawcols_train[0])
# DEL:
print(dev_rawcols_train[0])
print(len(represent_color_context(dev_rawcols_train[0])))
print(len(represent_color_context(dev_rawcols_train[0])[0]))

[[0.19444444444444445, 0.5, 0.11], [0.7472222222222222, 0.5, 0.27], [0.2722222222222222, 0.5, 0.73]]
3
54


__Your task__: Modify `represent_color_context` and/or `represent_color` to represent colors in a new way.
    
__Notes__:

* You are not required to keep `represent_color`. This might be unnatural if you want to perform an operation on each color trio all at once.
* For that matter, if you want to process all of the color contexts in the entire data set all at once, that is fine too, as long as you can also perform the operation at test time with an unknown number of examples being tested.

* The Fourier-transform method of [Monroe et al. 2016](https://www.aclweb.org/anthology/D16-1243/) and [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142) is a proven choice for our task. __It is not required that you implement this.__ However, if you decide to, you might find that the overly terse presentation in the paper is an obstacle. They key thing to see is that the notation $\hat{f}_{jkl}$ is meant to specify a full coordinate system. Thus, you might do something like

  ```
from itertools import product
for j, k, l in product((0, 1, 2), repeat=3):    
    f_jkl = ...
```

  and collect these `f_jkl` values in a list of 27 values. Additionally, in Python, [`2j` produces a value with `real` and `imag` attributes](https://docs.python.org/3.7/library/cmath.html). Each element `f_jkl` should have these components. If you concatenate the `real` and `imag` parts of all the `f_jkl`, you will have a 54-dimensional representation, as in the paper. Remember to start with an HSV representation, and with $h$ in $[0, 360]$, $s$ in $[0, 200]$, and $v$ in $[0, 200]$ (or else do the scaling differently). Note that the values in our corpus are in HLS format, [which are easily converted to HSV](https://en.wikipedia.org/wiki/HSL_and_HSV#HSV_to_HSL).
  
* It's natural to ask why this Fourier transform is useful in the current context. This is a challenging question, and I don't have a complete answer, but here is an intuitive observation: if you consider the raw color representations to be embeddings, then you can see very quickly that our standard geometric notions are totally out of line with our intuitions about the colors themselves. For example, here is a plot where we simply vary the hue dimension while keeping the other dimensions constant:

  <img src="fig/colors-hue-hls.png" alt="A series of very different colors with cosine distances from orange ranging from 0 to 0.19" />

  I've printed the cosine distances from the leftmost color above each patch. They all look pretty similar. Now, you might say, well at least the distances are sort of proportional to how different the colors are from the first. However, that argument seems to crumble when we do the same experiment but now varying the saturation dimension:

  <img src="fig/colors-saturation-hls.png" alt="A series of very similar purple-ish colors with cosine distances from gray-purple ranging from 0 to 0.19" />

  These colors are all quite simular intuitively. Notice, though, that the cosine distances are identical to my first plot. Of course! Cosine distances doesn't care about the nature of these dimensions! The underlying color space is a cylinder, not a regular Euclidean 3d space!
  
  The Fourier transformation that we apply is remapping the colors into approximately the cylindrical space that we want. It is at least capturing some the circular/radial relationships that are inherent in the space. Thus, here are plots corresponding to the above, but now where the colors have been transformed for the cosine comparisons. 
  
  First, the hue variation:
  
  <img src="fig/colors-hue-fourier.png" alt="A series of very different colors with cosine distances from orange that are generally large (near 1.0)" />

  And then saturation:
  
  <img src="fig/colors-saturation-fourier.png" alt="A series of very similar purple-ish colors with cosine distances from gray purple that seem aligned with visual color similarity" />
  
  These distances seem much better aligned with intuitions to me, and I think that's quite general. Thus, even if our networks can in principle learn this remapping, it's very helpful to at least start them closer to where we want them to be.
  
  If you want to go one layer deeper, then the [Zhang and Lu 2002](https://www.sciencedirect.com/science/article/pii/S092359650200084X) paper that Monroe et al. 2016 cite is pretty intuitive. It's for the 2d case, but that actually makes the ideas somewhere more accessible, since they can easily plot the original and remapped feature spaces.

The following test seeks to ensure only that the output of your `represent_color_context` will be compatible with the models we are creating:

In [145]:
def test_represent_color_context(func):
    """`func` should be `represent_color_context`"""
    example = [
        [0.786, 0.58, 0.87],
        [0.689, 0.44, 0.92],
        [0.628, 0.32, 0.81]]
    result = func(example)
    assert len(result) == len(example), \
        ("Color context representations need to represent each color "
         "separately. (We assume the final color is the target.)")
    for i, color in enumerate(result):
        assert all(isinstance(x, float) for x in color), \
            ("All color representations should be lists of floats. "
             "Color {} is {}".format(i, color))

In [42]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_represent_color_context(represent_color_context)

## Use the color representer

The following cell just runs your `represent_color_context` on the train and test sets:

In [195]:
dev_cols_train = [represent_color_context(colors) for colors in dev_rawcols_train]

dev_cols_test = [represent_color_context(colors) for colors in dev_rawcols_test]

At this point, our preprocessing steps are complete, and we can fit a first model.

## Initial model

The first model is configured right now to be a small model run for just a few iterations. It should be enough to get traction, but it's unlikely to be a great model. You are free to modify this configuration if you wish; it is here just for demonstration and testing:

In [196]:
dev_mod = ContextualColorDescriber(
    dev_vocab,
    early_stopping=True)

In [197]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    %time _ = dev_mod.fit(dev_cols_train, dev_seqs_train)
else:
    dev_mod.fit(dev_cols_train, dev_seqs_train)

/Users/miraleung/scu/xcs224u/hw/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 1 of 1000; error is 67.94107341766357/Users/miraleung/scu/xcs224u/hw/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Finished epoch 2 of 1000; error is 66.17325782775879/Users/miraleung/scu/xcs224u/hw/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]
Stopping after epoch 15. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 52.98034334182739

CPU times: user 2min 22s, sys: 42 s, total: 3min 4s
Wall time: 1min 15s


The canonical bake-off evaluation function is `evaluate`. Our primary metric is `listener_accuracy`; the BLEU score is included as a check to ensure that your system is speaking English!

In [198]:
evaluation = dev_mod.evaluate(dev_cols_test, dev_seqs_test)

In [79]:
evaluation.keys()

dict_keys(['listener_accuracy', 'corpus_bleu', 'target_index', 'predicted_index', 'predicted_utterance'])

In [199]:
evaluation['listener_accuracy']

0.704290238986467

In [29]:
dev_mod.listener_accuracy(dev_cols_test, dev_seqs_test)

0.7849121796717535

In [230]:
evaluation['corpus_bleu']

0.2766748802438902

In [229]:
bleu, predicted_utterances = dev_mod.corpus_bleu(dev_cols_test, dev_seqs_test)

bleu

0.07809143394199784

In [32]:
evaluation['target_index'][: 5]

[2, 2, 2, 2, 2]

In [33]:
evaluation['predicted_index'][: 5]

[2, 2, 2, 2, 2]

In [34]:
evaluation['predicted_utterance'][: 5]

[['<s>', 'dull', 'green', '</s>'],
 ['<s>', 'green', 'ish', 'yellow', '</s>'],
 ['<s>', 'dark', 'blue', '</s>'],
 ['<s>', 'yellow', 'green', '</s>'],
 ['<s>', 'dark', 'purple', '</s>']]

We can also see the model's predicted sequences given color context inputs:

In [35]:
dev_mod.predict(dev_cols_test[: 1])

[['<s>', 'dull', 'green', '</s>']]

In [36]:
dev_seqs_test[: 1]

[['<s>', 'green', 'ish', 'brown', '</s>']]

## Question 3: GloVe embeddings [1 point]

The above model uses a random initial embedding, as configured by the decoder used by `ContextualColorDescriber`. This homework question asks you to consider using GloVe inputs. 

__Your task__: Complete `create_glove_embedding` so that it creates a GloVe embedding based on your model vocabulary. This isn't mean to be analytically challenging, but rather just to create a basis for you to try out other kinds of rich initialization.

In [200]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [201]:
def create_glove_embedding(vocab, glove_base_filename='glove.6B.50d.txt'):
    pass
    # Use `utils.glove2dict` to read in the GloVe file:
    d = utils.glove2dict(os.path.join(GLOVE_HOME, glove_base_filename))

    # Use `utils.create_pretrained_embedding` to create the embedding.
    # This function will, by default, ensure that START_TOKEN,
    # END_TOKEN, and UNK_TOKEN are included in the embedding.

    # Be sure to return the embedding you create as well as the
    # vocabulary returned by `utils.create_pretrained_embedding`,
    # which is likely to have been modified from the input `vocab`.
    return utils.create_pretrained_embedding(d, vocab)

In [202]:
def test_create_glove_embedding(func):
    vocab = ['NLU', 'is', 'the', 'future', '.', '$UNK', '<s>', '</s>']
    glove_embedding, glove_vocab = func(vocab, 'glove.6B.50d.txt')
    assert isinstance(glove_embedding, np.ndarray), \
        "Expected embedding type {}; got {}".format(
        glove_embedding.__class__.__name__, glove_embedding.__class__.__name__)
    assert glove_embedding.shape == (8, 50), \
        "Expected embedding shape (8, 50); got {}".format(glove_embedding.shape)
    assert glove_vocab == vocab, \
        "Expected vocab {}; got {}".format(vocab, glove_vocab)

In [51]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_create_glove_embedding(create_glove_embedding)

## Try the GloVe representations

The extent to which GloVe is useful will depend heavily on how aligned your tokenization scheme is with the GloVe vocabulary. For example, if you did character-level tokenization, then the GloVe embedding space is not well-aligned with your tokenizer and using GloVe should have little no positive effect.

Let's see if GloVe helped for our development data:

In [204]:
dev_glove_embedding, dev_glove_vocab = create_glove_embedding(dev_vocab)

In [40]:
len(dev_vocab)

1180

In [41]:
len(dev_glove_vocab)

1180

In [205]:
dev_mod_glove = ContextualColorDescriber(
    dev_glove_vocab,
    embedding=dev_glove_embedding,
    early_stopping=True)

In [206]:
_ = dev_mod_glove.fit(dev_cols_train, dev_seqs_train)

Stopping after epoch 15. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 53.71348333358765

In [207]:
dev_mod_glove.listener_accuracy(dev_cols_test, dev_seqs_test)

0.7797293406276994

You probably saw a small boost, assuming your tokeization scheme leads to good overlap with the GloVe vocabulary. The input representations are larger than in our previous model (at least as I configured things), so we would need to do more runs with higher `max_iter` values to see whether this is worthwhile overall.

## Question 4: Color context [3 points]

The final required homework question is the most challenging, but it should set you up to think in much more flexible ways about the underlying model we're using.

The question asks you to modify various model components in `torch_color_describer.py`. The section called [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model) from the core unit notebook provides a number of examples illustrating the basic techniques, so you might review that material if you get stuck here.

__Your task__: Building on ideas from [Monroe et al. 2017](https://transacl.org/ojs/index.php/tacl/article/view/1142), you will redesign the model so that the target color (the final one in the context) is appended to each input token that gets processed by the decoder. The question asks you to subclass the `Decoder` and `EncoderDecoder` from `torch_color_describer.py` so that you can build models that do this.

__Step 1__: Modify the `Decoder` so that the input vector to the model at each timestep is not just a token representation `x` but the concatenation of `x` with the representation of the target color.

__Notes__:

* You might notice at this point that the original `Decoder.forward` method has an optional keyword argument `target_colors` that is passed to `Decoder.get_embeddings`. Because this is already in place, all you have to do is modify the `get_embeddings` method to use this argument.

* The change affects the configuration of `self.rnn`, so you need to subclass the `__init__` method as well, so that its `input_size` argument accomodates the embedding as well as the color representations.

* You can do the relevant operations efficiently in pure PyTorch using `repeat_interleave` and `cat`, but the important thing is to get a working implementation – you can always optimize the code later if the ideas prove useful to you. 

Here's skeleton code for you to flesh out:

In [208]:
from torch_color_describer import Decoder
import torch
import torch.nn as nn


class ColorContextDecoder(Decoder):
    def __init__(self, color_dim, *args, **kwargs):
        self.color_dim = color_dim
        super().__init__(*args, **kwargs)

        # Fix the `self.rnn` attribute.
        self.rnn = nn.GRU(
            input_size=self.embed_dim + self.color_dim,
            hidden_size=self.hidden_dim,
            batch_first=True)


    def get_embeddings(self, word_seqs, target_colors=None):
        """
        You can assume that `target_colors` is a tensor of shape
        (m, n), where m is the length of the batch (same as
        `word_seqs.shape[0]`) and n is the dimensionality of the
        color representations the model is using. The goal is
        to attached each color vector i to each of the tokens in
        the ith sequence of (the embedded version of) `word_seqs`.
        """
        _, num_repeated = word_seqs.shape
        colors_repeated = torch.repeat_interleave(target_colors, num_repeated, dim=0)
        return torch.cat((
            self.embedding(word_seqs),
            torch.reshape(colors_repeated, (target_colors.shape[0], num_repeated, target_colors.shape[1]))),
            2)


Step 1 is the most demanding of the steps in terms of tensor wrangling. It's important to have a clear idea of what you are trying to achieve and to unit test `get_embeddings` so that you can check that it has realized your vision. The following test should help with that:

In [183]:
def test_get_embeddings(decoder_class):
    """
    It's assumed that the input to this will be `ColorContextDecoder`.
    You pass in the class, and the function initalizes it with the test
    parameters.
    """
    dec = decoder_class(
        color_dim=3,   # For these, we mainly want *different*
        vocab_size=10, # dimensions so that we reliably get
        embed_dim=4,   # dimensionality errors if something
        hidden_dim=5)  # isn't working.

    # This step just changes the embedding to one with values
    # that are easy to inspect and definitely will not change
    # between runs:
    dec.embedding = nn.Embedding.from_pretrained(
        torch.FloatTensor([
            [10, 11, 12, 13],
            [14, 15, 16, 17],
            [18, 19, 20, 21]]))

    # These are the incoming sequences -- lists of indices
    # into the rows of `dec.embedding`:
    word_seqs = torch.tensor([
        [0,1,2],
        [2,0,1]])

    # Target colors as small floats that will be easy to track:
    target_colors = torch.tensor([
        [0.1, 0.2, 0.3],
        [0.7, 0.8, 0.9]])

    # The desired return value: one list of tensors for each of
    # the two sequences in `word_seqs`. Each index is replaced
    # with its vector from `dec.embedding` and has the
    # corrresponding color from `target_colors` appended to it.
    expected = torch.tensor([
        [[10., 11., 12., 13.,  0.1,  0.2,  0.3],
         [14., 15., 16., 17.,  0.1,  0.2,  0.3],
         [18., 19., 20., 21.,  0.1,  0.2,  0.3]],

        [[18., 19., 20., 21.,  0.7,  0.8,  0.9],
         [10., 11., 12., 13.,  0.7,  0.8,  0.9],
         [14., 15., 16., 17.,  0.7,  0.8,  0.9]]])

    result = dec.get_embeddings(word_seqs, target_colors=target_colors)

    assert expected.shape == result.shape, \
        "Expected shape {}; got shape {}".format(expected.shape, result.shape)

    assert torch.all(expected.eq(result)), \
        ("Your result has the desired shape but the values aren't correct. "
         "Here's what your function creates; compare it with `expected` "
         "from the test:\n{}".format(result))

In [45]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_get_embeddings(ColorContextDecoder)

__Step 2__: Modify the `EncoderDecoder`. For this, you just need to make a small change to the `forward` method: extract the target colors from `color_seqs` and feed them to the decoder.

In [209]:
from torch_color_describer import EncoderDecoder

class ColorizedEncoderDecoder(EncoderDecoder):

    def forward(self,
            color_seqs,
            word_seqs,
            seq_lengths=None,
            hidden=None,
            targets=None):
        if hidden is None:
            hidden = self.encoder(color_seqs)

        # Extract the target colors from `color_seqs` and
        # feed them to the decoder, which already has a
        # `target_colors` keyword.
        num_contexts = color_seqs.shape[1]
        target_colors = color_seqs[:, num_contexts - 1, :]
        output, hidden = self.decoder(
            word_seqs,
            seq_lengths=seq_lengths,
            hidden=hidden,
            target_colors=target_colors)

        # Your decoder will return `output, hidden` pairs; the
        # following will handle the two return situations that
        # the code needs to consider -- training and prediction.
        if self.training:
            return output
        else:
            return output, hidden

__Step 3__: Finally, as in the examples in [Modifying the core model](colors_overview.ipynb#Modifying-the-core-model), you need to modify the `build_graph` method of `ContextualColorDescriber` so that it uses your new `ColorContextDecoder` and `ColorizedEncoderDecoder`. Here's starter code:

In [210]:
from torch_color_describer import Encoder

class ColorizedInputDescriber(ContextualColorDescriber):

    def build_graph(self):

        # We didn't modify the encoder, so this is
        # just copied over from the original:
        encoder = Encoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim)

        # Use your `ColorContextDecoder`, making sure
        # to pass in all the keyword arguments coming
        # from `ColorizedInputDescriber`:
        decoder = ColorContextDecoder(
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            hidden_dim=self.hidden_dim,
            embedding=self.embedding)

        # Return a `ColorizedEncoderDecoder` that uses
        # your encoder and decoder:
        return ColorizedEncoderDecoder(encoder, decoder)

That's it! Since these modifications are pretty intricate, you might want to use [a toy dataset](colors_overview.ipynb#Toy-problems-for-development-work) to debug it:

In [211]:
def test_full_system(describer_class):
    toy_color_seqs, toy_word_seqs, toy_vocab = create_example_dataset(
        group_size=50, vec_dim=2)

    toy_color_seqs_train, toy_color_seqs_test, toy_word_seqs_train, toy_word_seqs_test = \
        train_test_split(toy_color_seqs, toy_word_seqs)

    toy_mod = describer_class(toy_vocab)

    _ = toy_mod.fit(toy_color_seqs_train, toy_word_seqs_train)

    acc = toy_mod.listener_accuracy(toy_color_seqs_test, toy_word_seqs_test)

    return acc

In [61]:
test_full_system(ColorizedInputDescriber)

Finished epoch 1000 of 1000; error is 0.09947841614484787

1.0

If that worked, then you can now try this model on SCC problems!

## Your original system [3 points]

There are many options for your original system, which consists of the full pipeline – all preprocessing and modeling steps. You are free to use any model you like, as long as you subclass `ContextualColorDescriber` in a way that allows its `evaluate` method to behave in the expected way.

So that we can evaluate models in a uniform way for the bake-off, we ask that you modify the function `evaluate_original_system` below so that it accepts a trained instance of your model and does any preprocessing steps required by your model.

If we seek to reproduce your results, we will rerun this entire notebook. Thus, it is fine if your `evaluate_original_system` makes use of functions you wrote or modified above this cell.

In [212]:
# Original System
# TODO: ADD DESCRIPTION HERE

import torch.nn as nn
from torch_color_describer import Encoder, Decoder, EncoderDecoder

class DeepEncoder(Encoder):
    def __init__(self, *args, num_layers=2, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_layers = num_layers
        self.rnn = nn.GRU(
        #self.rnn = nn.LSTM(
            input_size=self.color_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True)

class DeepDecoder(Decoder):
    def __init__(self, color_dim, num_layers=2, *args, **kwargs):
        self.color_dim = color_dim
        self.num_layers = num_layers
        super().__init__(*args, **kwargs)

        # Fix the `self.rnn` attribute.
        #self.rnn = nn.LSTM(
        self.rnn = nn.GRU(
            input_size=self.embed_dim + self.color_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True)


    def get_embeddings(self, word_seqs, target_colors=None):
        """
        You can assume that `target_colors` is a tensor of shape
        (m, n), where m is the length of the batch (same as
        `word_seqs.shape[0]`) and n is the dimensionality of the
        color representations the model is using. The goal is
        to attached each color vector i to each of the tokens in
        the ith sequence of (the embedded version of) `word_seqs`.
        """
        _, num_repeated = word_seqs.shape
        colors_repeated = torch.repeat_interleave(target_colors, num_repeated, dim=0)
        return torch.cat((
            self.embedding(word_seqs),
            torch.reshape(colors_repeated, (target_colors.shape[0], num_repeated, target_colors.shape[1]))),
            2)


class DeepContextualColorDescriber(ContextualColorDescriber):
    def __init__(self, vocab, num_layers=5, *args, **kwargs):
        self.num_layers = num_layers
        super().__init__(vocab, *args, **kwargs)
        # DEL:
        #self.hidden_dim=10
        #self.embed_dim=10      
        print("Config: num_layers: {0}, hidden_dim: {1}, embed_dim={2}".format(
            self.num_layers, self.hidden_dim, self.embed_dim))

    def build_graph(self):
        encoder = DeepEncoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim,
            num_layers=self.num_layers)  # The new piece is this argument.

        decoder = DeepDecoder(
            color_dim=self.color_dim,
            num_layers=self.num_layers,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim)

        return ColorizedEncoderDecoder(encoder, decoder)
    

class CvEncoder(Encoder):
    def __init__(self, *args, padding=0, **kwargs):
        super().__init__(*args, **kwargs)
        self.padding = padding
        self.kernel_size = 3
        self.conv1 = nn.Conv1d(
            self.color_dim, self.hidden_dim, kernel_size=self.kernel_size, padding=self.padding)

    def forward(self, color_seqs):
        h = self.conv1(color_seqs.permute(0, 2, 1))
        hidden, _ = torch.max(h, dim=-1)
        return hidden.unsqueeze(dim=0)
    
class CvColorDescriber(ContextualColorDescriber): 
    def __init__(self, *args, padding=0, **kwargs):
        self.padding = padding
        super().__init__(*args, **kwargs)
        
                
    def build_graph(self):
        encoder = CvEncoder(
            color_dim=self.color_dim,
            hidden_dim=self.hidden_dim,
            padding=self.padding
        ) 
        
        """
        decoder = DeepDecoder(
            color_dim=self.color_dim,
            num_layers=1,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim)
        
        """
        decoder = ColorContextDecoder(
            color_dim=self.color_dim,
            vocab_size=self.vocab_size,
            embed_dim=self.embed_dim,
            embedding=self.embedding,
            hidden_dim=self.hidden_dim)
        return ColorizedEncoderDecoder(encoder, decoder)

In [63]:
test_full_system(DeepContextualColorDescriber)

Finished epoch 5 of 1000; error is 1.5640995502471924

Config: num_layers: 5, hidden_dim: 50, embed_dim=50


Finished epoch 1000 of 1000; error is 0.10798472911119461

1.0

In [223]:
# DEL: More testing

"""
dev_mod_deep = DeepContextualColorDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    num_layers=5,
    embed_dim=10, 
    hidden_dim=10, 
    max_iter=25, 
    batch_size=128)
_ = dev_mod_deep.fit(dev_cols_train, dev_seqs_train)
dev_mod_deep_acc = dev_mod_deep.listener_accuracy(dev_cols_test, dev_seqs_test)
print("Accuracy: {0}".format(dev_mod_deep_acc))
"""
dev_mod_cv = CvColorDescriber(
    dev_glove_vocab, 
    embedding=dev_glove_embedding,
    hidden_dim=5, 
    embed_dim=50,
    max_iter=5, 
    batch_size=256)
_ = dev_mod_cv.fit(dev_cols_train, dev_seqs_train)
dev_mod_cv_acc = dev_mod_cv.listener_accuracy(dev_cols_test, dev_seqs_test)
print("Accuracy: {0}".format(dev_mod_cv_acc))

Finished epoch 5 of 5; error is 219.24215126037598/Users/miraleung/scu/xcs224u/hw/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]


Accuracy: 0.9009501871580766


# DEL: Data / Testing

Used: 
- Conv1d, hidden=5, embed=50, batch_size=256, max_iter=5

## BERT tokenizer only, raw, bert-large-uncased
- Conv1d, hidden=5, embed=50, batch_size=256, max_iter=5
  - Finished epoch 5 of 5; error is 227.30396032333374
    - Accuracy: 0.9021019291678664 
- GRU, 5 layers, hidden=10, embed=10, DeepEncoder, max_iter=25, batch_size=128
  - Finished epoch 25 of 25; error is 241.92416071891785
    - Accuracy: 0.636049524906421

## BERT tokenizer only, raw, bert-large-cased

## BERT tokenizer only, raw, bert-base-cased
- Conv1d, hidden=5, embed=50, batch_size=256, max_iter=5
  - Accuracy: 0.8856896055283616
- GRU, 5 layers, hidden=10, embed=10, DeepEncoder, max_iter=25, batch_size=128
  - Finished epoch 25 of 25; error is 255.03439044952393
    - Accuracy: 0.6308666858623668

## BERT tokenizer only, raw, bert-base-uncased
BEST: Conv1d, hidden=5, embed=50, batch_size=256, max_iter=5
BEST SCORE: Accuracy: 0.937517995968903

- Conv1d, hidden=5, embed=45, batch_size=256, max_iter=5
  - Acc: 0.89
- Conv1d, hidden=5, embed=75, batch_size=256, max_iter=5
  - Accuracy: 0.8972070256262598 
- Conv1d, hidden=5, embed=25, batch_size=256, max_iter=5
  - Accuracy: 0.9015260581629715
- Conv1d, hidden=5, embed=100, batch_size=256, max_iter=5
  - Finished epoch 5 of 5; error is 235.12957191467285
    - Accuracy: 0.9136193492657645
- Conv1d, hidden=5, embed=50, batch_size=256, max_iter=5
  - Finished epoch 5 of 5; error is 235.91493177413943
    - Accuracy: 0.937517995968903 
- Conv1d, hidden=10, embed=50, batch_size=256, max_iter=5
  - Finished epoch 5 of 5; error is 221.97119903564453
    - Accuracy: 0.9271523178807947 
- Conv1d, hidden=10, embed=10, batch_size=256, max_iter=5
  - Finished epoch 5 of 5; error is 225.48944616317756
    - Accuracy: 0.9064209617045782 
- Conv1d, hidden=10, embed=50, batch_size=1024, max_iter=5
  - Finished epoch 5 of 5; error is 73.37850093841553
    - Accuracy: 0.8468183126979557
- Conv1d, hidden=10, embed=50, batch_size=512, max_iter=5
  - Finished epoch 5 of 5; error is 127.20562791824341
    - Accuracy: 0.9234091563489778
- Conv1d, hidden=10, embed=50, batch_size=128, max_iter=5
  - Finished epoch 5 of 5; error is 376.87632679939275
    - Accuracy: 0.9023898646703139
- Conv1d, hidden=10, embed=50, batch_size=128, max_iter=10
  - Finished epoch 10 of 10; error is 333.52702474594116
    - Accuracy: 0.9098761877339476
- Conv1d, hidden=10, embed=50, batch_size=128, max_iter=25
  - Finished epoch 25 of 25; error is 209.70607900619507
    - Accuracy: 0.9072847682119205
- Conv1d, hidden=10, embed=10, batch_size=128, max_iter=25
  - Finished epoch 25 of 25; error is 225.60237765312195
    - Accuracy: 0.9072847682119205 
- Conv1d, hidden=100, embed=100, batch_size=128, max_iter=25
  - Finished epoch 25 of 25; error is 189.2860366106033
    - Accuracy: 0.8180247624532104 
- GRU, 5 layers, hidden=10, embed=10, DeepEncoder, max_iter=25, batch_size=128
  - Finished epoch 25 of 25; error is 244.73929738998413
    - Accuracy: 0.6947883674057012 
- GRU, 5 layers, hidden=10, embed=10, DeepEncoder, max_iter=5, batch_size=128
  - Finished epoch 5 of 5; error is 417.18788480758667
    - Accuracy: 0.49784048373164413 
- GRU, 5 layers, hidden=5, embed=50, DeepEncoder, max_iter=5, batch_size=256
  - Finished epoch 5 of 5; error is 221.77516222000122
    - Accuracy: 0.4644399654477397 

## BERT tokenizer only, to-lower, bert-large-uncased
- Conv1d, hidden=100, embed=100, batch_size=128, max_iter=25
  - Finished epoch 25 of 25; error is 187.47414970397954
    - Accuracy: 0.8131298589116038 
- GRU, 5 layers, hidden=10, embed=10, DeepEncoder, max_iter=25, batch_size=128
- LSTM, 5 layers, hidden=10, embed=10, DeepEncoder, max_iter=25, batch_size=128
  - Finished epoch 25 of 25; error is 249.21498870849616
    - Accuracy: 0.44226893175928594

## New BERT (bert-large-uncased) + to-lower + Monroe tokenizer
- Conv1d, hidden=100, embed=100, batch_size=128, max_iter=25
  - Finished epoch 25 of 25; error is 180.60007059574127
    - Accuracy: 0.8016124388137057 
- GRU, 5 layers, hidden=10, embed=10, DeepEncoder, max_iter=25, batch_size=128
  - Finished epoch 25 of 25; error is 229.81808829307556
    - Accuracy: 0.5928591995393032
- LSTM, 5 layers, hidden=10, embed=10, DeepEncoder, max_iter=25, batch_size=128
  - Finished epoch 25 of 25; error is 231.06814122200012
    - Accuracy: 0.6052404261445437


## Old Monroe, Stanford parser

- LSTM, 5 layers, hidden=50, embed=50
  - Finished epoch 1000 of 1000; error is 0.11445938050746918 
- GRU, 2 layers, hidden=50, embed=50
  - Finished epoch 1000 of 1000; error is 0.12016019970178604
- GRU, 5 layers, hidden=50, embed=50
  - Finished epoch 1000 of 1000; error is 0.10635438561439514
  - Finished epoch 1000 of 1000; error is 0.10232776403427124
- GRU, 5 layers, hidden=10, embed=10
  - Finished epoch 1000 of 1000; error is 0.10539861768484116
- GRU, 5 layers, hidden=250, embed=10
  - Finished epoch 1000 of 1000; error is 0.10099897533655167 
- GRU, 5 layers, hidden=5, embed=5
  - Finished epoch 1000 of 1000; error is 0.25616884231567383
  

## Encoder type
- GRU, 10 layers, hidden=300, embed=300, DeepEncoder, batch 1024
  - Finished epoch 5 of 5; error is 56.547504425048836
- GRU, 5 layers, hidden=300, embed=50, DeepEncoder, batch 1024
  - Finished epoch 5 of 5; error is 57.317170619964695
  - Accuracy: 0.33659660236107114 
- GRU, 5 layers, hidden=300, embed=300, DeepEncoder, batch 1024
  - Finished epoch 5 of 5; error is 56.104586601257324 
- GRU, 5 layers, hidden=100, embed=300, DeepEncoder
  - Finished epoch 5 of 5; error is 58.845255851745605
- GRU, 5 layers, hidden=10, embed=10, DeepEncoder, batch 256
  - Finished epoch 5 of 5; error is 211.65790700912476
  - Accuracy: 0.3253671177656205 
  - Finished epoch 5 of 5; error is 218.40114831924438
  - Accuracy: 0.3210480852289087
  
- GRU, 5 layers, hidden=10, embed=10, DeepEncoder
  - Finished epoch 5 of 5; error is 225.49082040786743 
  - Batch 1024: Finished epoch 5 of 5; error is 74.06650495529175 
- GRU, 5 layers, hidden=10, embed=100, DeepEncoder
  - Finished epoch 5 of 5; error is 73.15999841690063
- GRU, 5 layers, hidden=10, embed=50, DeepEncoder
  - Batch 256: Finished epoch 5 of 5; error is 218.55037784576416
  - Batch 512: 128.20260238647467
  - Batch 1024: Finished epoch 5 of 5; error is 73.60409688949585 
- Conv1d, hidden=10, embed=50 (default), batch_size=256, max_iter=5
  - Finished epoch 5 of 5; error is 202.05388069152832
- Conv1d, hidden=10, embed=10, batch_size=256, max_iter=1000
  - Finished epoch 80 of 1000; error is 62.842184662818918
- Conv1d, hidden=10, embed=10, batch_size=128, max_iter=10
  - Finished epoch 15 of 1000; error is 266.63916587829594
- Conv1d, hidden=10, embed=default, batch_size=128, max_iter=10, GRU decoder
  - Finished epoch 5 of 5; error is 328.58452320098877
- Conv1d, hidden=10, embed=50 (default), batch_size=128, max_iter=5
  - Finished epoch 5 of 5; error is 351.91490721702576
  - Accuracy: 0.5873884249928016
- Conv1d, hidden=100, embed=50 (default), batch_size=1024, max_iter=5
  - Finished epoch 5 of 5; error is 57.56720161437988
  - Accuracy: 0.6003455226029369 
- Conv1d, hidden=10, embed=50 (default), batch_size=1024, max_iter=5
  - Finished epoch 5 of 5; error is 74.23692989349365
  - Accuracy: 0.42182551108551686
- Conv1d, hidden=10, embed=50 (default), batch_size=256, max_iter=5
  - Finished epoch 5 of 5; error is 203.10836172103882
  - Accuracy: 0.5295133890008638
- Conv1d, hidden=10, embed=10 (default), batch_size=128, max_iter=5
  - Finished epoch 5 of 5; error is 365.06503653526306
  - Accuracy: 0.5885401670025914
- Conv1d, hidden=10, embed=10, batch_size=128, max_iter=25
  - Finished epoch 25 of 25; error is 226.11915516853333
  - Accuracy: 0.7569824359343507
- Conv1d, hidden=100, embed=100, batch_size=128, max_iter=25
  - Finished epoch 25 of 25; error is 173.57468342781067
  - Accuracy: 0.785200115174201

In [85]:
# Train
# DEL: TODO: Hyperparam tuning, param passing, try without glove
mod_bakeoff = DeepContextualColorDescriber(
    dev_glove_vocab,
    num_layers=5,
    embedding=dev_glove_embedding,
    early_stopping=True, 
    embed_dim=10, 
    hidden_dim=10)
%time _ = mod_bakeoff.fit(dev_cols_train, dev_seqs_train)

Stopping after epoch 35. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 49.37194013595581

CPU times: user 8min 27s, sys: 2min 5s, total: 10min 32s
Wall time: 5min 29s


In [224]:
mod_bakeoff = dev_mod_cv
dev_mod = mod_bakeoff

# Evaluation

In [225]:
def evaluate_original_system(trained_model, color_seqs_test, texts_test):
    """
    Feel free to modify this code to accommodate the needs of
    your system. Just keep in mind that it will get raw corpus
    examples as inputs for the bake-off.

    """
    # `word_seqs_test` is a list of strings, so tokenize each of
    # its elements:
    tok_seqs = [tokenize_example(s) for s in texts_test]

    col_seqs = [represent_color_context(colors)
                for colors in color_seqs_test]


    # Optionally include other preprocessing steps here. Note:
    # DO NOT RETRAIN YOUR MODEL AS PART OF THIS EVALUATION!
    # It's a tempting step, but it's a mistake and will get
    # you disqualified!

    # The following core score calculations are required:
    evaluation = trained_model.evaluate(col_seqs, tok_seqs)

    return evaluation

If `evaluate_original_system` works on test sets you create from the corpus distribution, then it will work for the bake-off, so consider checking that. For example, this would check that `dev_mod` above passes muster:

In [226]:
my_evaluation = evaluate_original_system(dev_mod, dev_rawcols_test, dev_texts_test)

/Users/miraleung/scu/xcs224u/hw/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]


In [227]:
my_evaluation['listener_accuracy']

0.9009501871580766

In [228]:
my_evaluation['corpus_bleu']

0.07809143394199784

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies. We also ask that you report the best **listener_accuracy** score your system got during development, just to help us understand how systems performed overall.

In [ ]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: MY_NUMBER
if 'IS_GRADESCOPE_ENV' not in os.environ:
    pass

# STOP COMMENT: Please do not remove this comment.

## Bakeoff [1 point]

For the bake-off, we will use our original test set. The function you need to run for the submission is the following, which uses your `evaluate_original_system` from above:

In [231]:
def create_bakeoff_submission(
        trained_model,
        output_filename='cs224u-colors-bakeoff-entry.csv'):
    bakeoff_src_filename = os.path.join(
        "data", "colors", "cs224u-colors-test.csv")

    bakeoff_corpus = ColorsCorpusReader(bakeoff_src_filename)

    # This code just extracts the colors and texts from the new corpus:
    bakeoff_rawcols, bakeoff_texts = zip(*[
        [ex.colors, ex.contents] for ex in bakeoff_corpus.read()])

    # Original system function call; `trained_model` is your trained model:
    evaluation = evaluate_original_system(
        trained_model, bakeoff_rawcols, bakeoff_texts)

    evaluation['bakeoff_text'] = bakeoff_texts

    df = pd.DataFrame(evaluation)
    df.to_csv(output_filename)

In [233]:
create_bakeoff_submission(dev_mod)

/Users/miraleung/scu/xcs224u/hw/cs224u/torch_color_describer.py:680: RuntimeWarning: divide by zero encountered in power
  perp = [np.prod(s)**(-1/len(s)) for s in scores]


This creates a file `cs224u-colors-bakeoff-entry.csv` in the current directory. That file should be uploaded as-is. Please do not change its name.

Only one upload per team is permitted, and you should do no tuning of your system based on what you see in the file – you should not study that file in anyway, beyond perhaps checking that it contains what you expected it to contain. The upload function will do some additional checking to ensure that your file is well-formed.

The nature of our evaluation is such that we have to release the full test set with all labels. Thus, we have to trust you not to make any use of the test set during development. Recall:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

Systems will be ranked primarily by `listener_accuracy`, but we will also consider their `corpus_bleu` scores. However, the BLEU score is just a simple check that your system is speaking some version of English that corresponds in some meaningful way to the gold descriptions, so you should concentrate on `listener_accuracy`.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points.

## Submission Instruction

Submit the following files to gradescope submission

- Please do not change the file name as described below
- `hw_colors.ipynb` (this notebook)
- `cs224u-colors-bakeoff-entry.csv` (bake-off output)